In [1]:
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
import os
os.getcwd()

'/home/alex/Solar-irradiance-Team08-IFT6759/Notebooks'

In [3]:
admin_config_path = "../train_cfg.json"
with open(admin_config_path, "r") as fd:
    admin_config = json.load(fd)

In [4]:
dataframe_path = admin_config['dataframe_path']
df = pd.read_pickle('../' + dataframe_path)

In [5]:
df.replace(to_replace="nan", value=np.NaN, inplace=True)
df.head()

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,BND_CLOUDINESS,BND_GHI,TBL_DAYTIME,...,GWN_CLOUDINESS,GWN_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,PSU_CLOUDINESS,PSU_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI,SXF_CLOUDINESS,SXF_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.500000,0.0,0.0,night,-1.500000,0.0,0.0,night,-3.800000
2010-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.493333,0.0,0.0,night,-1.446667,0.0,0.0,night,-3.893333
2010-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.420000,0.0,0.0,night,-1.766667,0.0,0.0,night,-4.166667
2010-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.533333,0.0,0.0,night,-2.033333,0.0,0.0,night,-4.040000
2010-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.586667,0.0,0.0,night,-2.006667,0.0,0.0,night,-4.133333


In [6]:
station_id = "BND"
image_type = "hdf5_8bit"
daytime = df[station_id + "_DAYTIME"] == 1.0
valid_ghi = ~df[station_id + "_GHI"].isna()
valid_datetime = daytime & valid_ghi
df = df.loc[valid_datetime]

In [7]:
# df.isna().sum()

In [8]:
candidate_datetimes = df.index
candidate_datetimes

DatetimeIndex(['2010-01-01 13:30:00', '2010-01-01 13:45:00',
               '2010-01-01 14:00:00', '2010-01-01 14:15:00',
               '2010-01-01 14:30:00', '2010-01-01 14:45:00',
               '2010-01-01 15:00:00', '2010-01-01 15:15:00',
               '2010-01-01 15:30:00', '2010-01-01 15:45:00',
               ...
               '2015-12-31 20:15:00', '2015-12-31 20:30:00',
               '2015-12-31 20:45:00', '2015-12-31 21:00:00',
               '2015-12-31 21:15:00', '2015-12-31 21:30:00',
               '2015-12-31 21:45:00', '2015-12-31 22:00:00',
               '2015-12-31 22:15:00', '2015-12-31 22:30:00'],
              dtype='datetime64[ns]', name='iso-datetime', length=106385, freq=None)

In [9]:
nb_images_for_training = 3
timedeltas = pd.timedelta_range(start='-15 min', periods=(nb_images_for_training-1), freq='-15 min')
print(candidate_datetimes[2])
print(candidate_datetimes[2] + timedeltas)

2010-01-01 14:00:00
DatetimeIndex(['2010-01-01 13:45:00', '2010-01-01 13:30:00'], dtype='datetime64[ns]', freq='-15T')


In [10]:
all(elem in candidate_datetimes for elem in (candidate_datetimes[2] + timedeltas))

True

In [11]:
retained_datetimes = []
for candidate_datetime in candidate_datetimes:
    if all(elem in candidate_datetimes for elem in (candidate_datetime + timedeltas)):
        retained_datetimes += [candidate_datetime]

In [12]:
print(len(candidate_datetimes))
print(len(retained_datetimes))

106385
101982


In [20]:
# retained_datetimes[:10]

In [65]:
# TODO: Check if images for the specified datetimes exist on disk
# for datetime in retained_datetimes:

# For each retained datetime Lookup in the dataframe which file to check.
target_datetime = df.loc[retained_datetimes[0],["hdf5_8bit_path","hdf5_8bit_offset"]]

In [66]:
hdf5_path = target_datetime["hdf5_8bit_path"]
hdf5_offset = target_datetime["hdf5_8bit_offset"]

# Temporary local implementation; should use server path; uses Jan 2010
local_path = "data/hdf5v7_8bit_Jan_2015/"
hdf5_path = local_path + hdf5_path.split("/")[-1]
print(hdf5_path)
# Temporary local implementation for df lookup; uses Jan 2015
hdf5_path = "../data/hdf5v7_8bit_Jan_2015/2015.01.01.0800.h5"
print(hdf5_path)

data/hdf5v7_8bit_Jan_2015/2010.01.01.0800.h5
../data/hdf5v7_8bit_Jan_2015/2015.01.01.0800.h5


In [60]:
os.getcwd()

'/home/alex/Solar-irradiance-Team08-IFT6759/Notebooks'

In [62]:
def check_data_ok(h5_data, hdf5_offset):
    for channel in ["ch1", "ch2", "ch3", "ch4", "ch6"]:
        dataset_lut_name = channel + "_LUT"
        if dataset_lut_name in h5_data and h5_data[dataset_lut_name][hdf5_offset] == -1:
            return False # Fail
    return True # Pass

In [64]:
import h5py
# Check if the file and offset exist on disk.

if os.path.isfile(hdf5_path):
    with h5py.File(hdf5_path, "r") as h5_data:
        data_ok = check_data_ok(h5_data, hdf5_offset)
        if data_ok:
            pass # write true in pandas DF